In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
train_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


# NOTE: 
**This code represents only a fraction of what I tried to get the best possible results. I recommend that you try other models, features, ideas to get an even better result**

# Exploratory data analysis(EDA)

In [3]:
train_data.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

ProfileReport - "The pandas_profiling library in Python include a method named as ProfileReport() which generate a basic report on the input DataFrame."

In [4]:


#CODE BELOW MAKES AUTOMATED REPORT WITH GRAPHS, I RECOMEND IT


# from pandas_profiling import ProfileReport
# train_profile = ProfileReport(train_data)
# train_profile

KNNImputer - "Imputation for completing missing values using k-Nearest Neighbors.
Each sample’s missing values are imputed using the mean value from n_neighbors nearest neighbors found in the training set. Two samples are close if the features that neither is missing are close."

In [4]:
from sklearn.impute import KNNImputer

train_data['Transported'] = train_data['Transported']*1
train_data['CryoSleep'] = train_data['CryoSleep']*1
test_data['CryoSleep'] = test_data['CryoSleep']*1

imputer = KNNImputer(n_neighbors=2, weights="uniform")


train_data[["Deck", "Num", "Side"]] = train_data["Cabin"].str.split("/", expand=True)
train_data[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'CryoSleep', 'Num']] = imputer.fit_transform(train_data[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'CryoSleep', 'Num']])



test_data[["Deck", "Num", "Side"]] = test_data["Cabin"].str.split("/", expand=True)
test_data[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'CryoSleep', 'Num']] = imputer.fit_transform(test_data[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'CryoSleep', 'Num']])


train_data['HomePlanet'].fillna("Earth", inplace=True)
train_data['Destination'].fillna("TRAPPIST-1e", inplace=True)
train_data['Deck'].fillna("F", inplace=True)
train_data['Side'].fillna("P", inplace=True)



test_data['HomePlanet'].fillna("Earth", inplace=True)
test_data['Destination'].fillna("TRAPPIST-1e", inplace=True)
test_data['Deck'].fillna("F", inplace=True)
test_data['Side'].fillna("P", inplace=True)

LabelEncoder - "Encode target labels with value between 0 and n_classes-1."

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data["Deck"]= le.fit_transform(train_data["Deck"])

le = LabelEncoder()
train_data["Num"]= le.fit_transform(train_data["Num"])

le = LabelEncoder()
train_data["Side"]= le.fit_transform(train_data["Side"])

le = LabelEncoder()
test_data["Deck"]= le.fit_transform(test_data["Deck"])

le = LabelEncoder()
test_data["Num"]= le.fit_transform(test_data["Num"])

le = LabelEncoder()
test_data["Side"]= le.fit_transform(test_data["Side"])

Make new feature called "AllSpending" 

In [6]:
train_data['AllSpending'] = train_data['RoomService'] + train_data['ShoppingMall'] + train_data['FoodCourt'] + train_data['Spa'] + train_data['VRDeck']
test_data['AllSpending'] = test_data['AllShop'] = test_data['RoomService'] + test_data['ShoppingMall'] + test_data['FoodCourt'] + test_data['Spa'] + test_data['VRDeck']

# Feature selection, scaling

In [7]:
from sklearn.preprocessing import StandardScaler

y = train_data["Transported"]


features = ["CryoSleep","RoomService","Spa","VRDeck","Deck","Side","AllSpending"]

cols_to_normalize = ["RoomService","Spa","VRDeck","AllSpending"]
scaler = StandardScaler()

train_data[cols_to_normalize ] = scaler.fit_transform(train_data[cols_to_normalize ])
test_data[cols_to_normalize ] = scaler.transform(test_data[cols_to_normalize ])

X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

To use optuna to get the find the most optimal parameters.

In [10]:
'''def catboost_objective(trial):
    #print("START__________________________________")
    param = {  
        
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"])
            }
        

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)
        
        
    skf = StratifiedKFold(n_splits=4)
    results=[]
    n_iterations=[]
    for train_index, test_index in skf.split(X, y):
        train_X, valid_X = X.iloc[train_index], X.iloc[test_index]
        train_y, valid_y = y.iloc[train_index], y.iloc[test_index]
        model = cb.CatBoostClassifier(**param).fit(train_X,train_y, verbose=0)
                                 
        #n_iteration = model.get_booster().best_iteration
        #n_iterations.append(n_iteration)
        result = accuracy_score(valid_y,(model.predict(valid_X)))
        results.append(result)
        #i=int(sum(n_iterations)/len(n_iterations))
   # print("Average n_ite=" + str(i))
   # print("% of scatter =" + str(np.std(n_iterations)/i))    
    n=sum(results)/len(results) 
    #print (n)
    #print("FINISH__________________________________\n")
    return n'''


In [16]:
'''import optuna
from sklearn.model_selection import StratifiedKFold
import catboost as cb

study_catboost = optuna.create_study(pruner=optuna.pruners.HyperbandPruner(),
                            direction='maximize')
study_catboost.optimize(catboost_objective, n_trials=1000)
print('Number of finished trials:', len(study_catboost.trials))
print('Best trial:', study_catboost.best_trial.params)'''

"import optuna\nfrom sklearn.model_selection import StratifiedKFold\nimport catboost as cb\n\nstudy_catboost = optuna.create_study(pruner=optuna.pruners.HyperbandPruner(),\n                            direction='maximize')\nstudy_catboost.optimize(catboost_objective, n_trials=1000)\nprint('Number of finished trials:', len(study_catboost.trials))\nprint('Best trial:', study_catboost.best_trial.params)"

# Model training

In [8]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

cat_model = CatBoostClassifier(verbose=0)
cat_model.fit(X, y)

y_pred = cat_model.predict(X)
print(confusion_matrix(y, y_pred))
asc=accuracy_score(y, y_pred)
print(asc)
predictions = cat_model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Transported': predictions})
output = output.replace({0: False, 1: True})

output.to_csv('submit_catboost_simple.csv', index=False)
print("Submission was successfully saved!")

[[3444  871]
 [ 548 3830]]
0.836765213390084
Submission was successfully saved!


Initial Kaggle score matched origional notebook of 0.81038. 